In [ ]:
!pip install timm
from timm import create_model
! pip install fastai --upgrade #restart kernel
from fastai.vision.all import *
dataset_path = Path('../input/petfinder-pawpularity-score')

**A lot of people in discussions seem to be having trouble in calculating accruate CV scores. So here I demnstrate how to do the same using inference on your saved models adapted from cdeotte's rapids svr pipeline.**

# Sturges Bins OOF Inference

In [ ]:
#creating train df for using st

set_seed(999, reproducible=True)
BATCH_SIZE = 8                       

train_df = pd.read_csv(dataset_path/'train.csv')
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

train_df['norm_score'] = train_df['Pawpularity']/100
train_df['norm_score']

seed=999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True
#Sturges' rule
num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
# num_bins

train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
train_df['fold'] = -1
N_FOLDS = 10        #was10
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')
train_df.fold.value_counts().plot.bar()

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

def get_data(fold):
    train_df_f = train_df.copy()  # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    dls = ImageDataLoaders.from_df(
                    train_df_f, #pass in train DataFrame
                   valid_col='is_valid', #
                   seed=999, #seed
                   fn_col='path', #filename/path is in the second column of the DataFrame
                   label_col='norm_score', #label is in the first column of the DataFrame
                   y_block=RegressionBlock, #The type of target
                   bs=BATCH_SIZE, #pass in batch size
                   num_workers=8,
                   item_tfms=Resize(384), #pass in item_tfms
                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(), Flip()]) #pass in batch_tfms
                            ) 
    
    return dls

def get_learner(fold_num):
    data = get_data(fold_num)
    model = create_model('swin_large_patch4_window12_384', pretrained=True, num_classes=data.c)
    rmse = AccumMetric(petfinder_rmse)
    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=[rmse]).to_fp16()
    return learn

In [ ]:
import gc
for i in range(1): #1 for demonstration substitute N_FOLDS for all oof files

    print(f'Fold {i} results')
    learn = get_learner(fold_num=i)
    if i<5: learn.path=Path('../input/sturges-bins-10fold-p1')   # download models 0-4 to {dir_name}/models
    else: learn.path=Path('../input/sturges-bins-10fold-p2')     # download models 5-9 to {dir_name}/models from colab/kernels or local machine

    #learn.model_dir=Path('models')
    learn = learn.load(f"model_fold_{i}")
    dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                            seed=999, #seed
                            fn_col='path', #filename/path is in the second column of the DataFrame
                            label_col='norm_score', #label is in the first column of the DataFrame
                            y_block=RegressionBlock, #The type of target
                            bs=BATCH_SIZE, #pass in batch size
                            num_workers=8,
                            item_tfms=Resize(384), #pass in item_tfms
                            batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(), Flip()])) 

    valid = train_df[train_df.fold == i].reset_index(drop=True) # VALID, OOF Preds 
    train_predictions,_ = learn.tta(dl=dls.test_dl(valid), n=7, beta=0)
    
    valid['Pawpularity'] = np.stack(train_predictions)*100
    valid.to_csv(f'oof_preds_sturges_fold_{i}.csv')
    
    del learn; torch.cuda.empty_cache(); gc.collect()


# Rice Bins OOF Inference

In [ ]:
#creating train df for using sturges bins

set_seed(365, reproducible=True)
BATCH_SIZE = 8                       

train_df = pd.read_csv(dataset_path/'train.csv')
train_df['path'] = train_df['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

train_df['norm_score'] = train_df['Pawpularity']/100
train_df['norm_score']

seed=365
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

import math

#Rice rule

num_bins = int(np.ceil(2*((len(train_df))**(1./3))))
num_bins

train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)


from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
train_df['fold'] = -1
N_FOLDS = 10        #was10
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')
train_df.fold.value_counts().plot.bar()

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

def get_data(fold):
    train_df_f = train_df.copy()  # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    dls = ImageDataLoaders.from_df(
                    train_df_f, #pass in train DataFrame
                   valid_col='is_valid', #
                   seed=365, #seed
                   fn_col='path', #filename/path is in the second column of the DataFrame
                   label_col='norm_score', #label is in the first column of the DataFrame
                   y_block=RegressionBlock, #The type of target
                   bs=BATCH_SIZE, #pass in batch size
                   num_workers=8,
                   item_tfms=Resize(384), #pass in item_tfms
                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(), Flip()]) #pass in batch_tfms
                            ) 
    
    return dls


def get_learner(fold_num):
    data = get_data(fold_num)
    model = create_model('swin_large_patch4_window12_384', pretrained=True, num_classes=data.c)
    rmse = AccumMetric(petfinder_rmse)
    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=[rmse]).to_fp16()
    return learn

In [ ]:
import gc
for i in range(1):

    print(f'Fold {i} results')
    learn = get_learner(fold_num=i)
    if i<5: learn.path=Path('../input/rice-bins-10fold-p1')   # download models 0-4 to {dir_name}/models, for me I used gdown to download models to a kernel and saved
    else: learn.path=Path('../input/rice-bins-10fold-p2')     # download models 5-9 to {dir_name}/models from colab/kernels or local machine

    #learn.model_dir=Path('models')
    learn = learn.load(f"model_fold_{i}")
    dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                            seed=365, #seed
                            fn_col='path', #filename/path is in the second column of the DataFrame
                            label_col='norm_score', #label is in the first column of the DataFrame
                            y_block=RegressionBlock, #The type of target
                            bs=BATCH_SIZE, #pass in batch size
                            num_workers=8,
                            item_tfms=Resize(384), #pass in item_tfms
                            batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(), Flip()])) 

    valid = train_df[train_df.fold == i].reset_index(drop=True) # VALID, OOF Preds 
    train_predictions,_ = learn.tta(dl=dls.test_dl(valid), n=7, beta=0)
    
    valid['Pawpularity'] = np.stack(train_predictions)*100
    valid.to_csv(f'oof_preds_rice_fold_{i}.csv')
    
    del learn; torch.cuda.empty_cache(); gc.collect()


# CV Scores using CSV OOF Preds files

In [ ]:
print("strurges bins individual CV Scores")
for i in range(10):
    train=pd.read_csv(f'../input/sturgesoof/oof_preds_fold_{i}.csv')
    oof = np.hstack(train['Pawpularity']/100)
    true = np.hstack(train.norm_score.values)
    rmse = np.sqrt( np.mean( (oof - true)**2.0 ))
    print(f'fold{i} rmse = ', rmse)
    
print("rice bins individual CV Scores")
for i in range(10):
    train=pd.read_csv(f'../input/rice-oof/oof_preds_fold_{i}.csv')
    oof = np.hstack(train['Pawpularity']/100)
    true = np.hstack(train.norm_score.values)
    rmse = np.sqrt( np.mean( (oof - true)**2.0 ))
    print(f'fold{i} rmse = ', rmse)

In [ ]:
print("strurges bins")
oof_preds = []
oof_true = []
for i in range(10):
    train=pd.read_csv(f'../input/sturgesoof/oof_preds_fold_{i}.csv')
    oof_preds.append(train['Pawpularity']/100); oof_true.append(train.norm_score.values) 

true = np.hstack(oof_true); oof = np.hstack(oof_preds)   
rmse = 100*np.sqrt( np.mean( (oof - true)**2.0 ))
print("overall CV", rmse)

print("rice bins")
oof_preds = []
oof_true = []
for i in range(10):
    train=pd.read_csv(f'../input/rice-oof/oof_preds_fold_{i}.csv')
    oof_preds.append(train['Pawpularity']/100); oof_true.append(train.norm_score.values) 

true = np.hstack(oof_true); oof = np.hstack(oof_preds)   
rmse = 100*np.sqrt( np.mean( (oof - true)**2.0 ))
print("overall CV", rmse)


In [ ]:
oof_preds = []; oof_true = []
train=pd.read_csv(f'./oof_preds_sturges_fold_0.csv')
oof_preds.append(train['Pawpularity']/100); oof_true.append(train.norm_score.values) 
true = np.hstack(oof_true); oof = np.hstack(oof_preds)   
rmse = 100*np.sqrt( np.mean( (oof - true)**2.0 ))

In [ ]:
rmse